In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
db_name = config['DEFAULT']['DB-Name']
collection_name = config['DEFAULT']['Collection-Name']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

import time

In [ ]:
# connect to database
db_twitter = client[db_name]

In [ ]:
# pipeline for aggregation
pipeline = [
    {"$match": { "entities.hashtags": {"$exists":True,"$ne":[]}}},
    {"$match": { "lang" : "en"}},
    { "$group": {
        "_id": {
            "hashtags": "$entities.hashtags",
            "date": {"$substr": [ "$created_at", 4, 6 ]}
        },
        "count": { "$sum": 1 },
        }
    }
]

In [ ]:
start = time.time()
data_list = list(db_twitter[collection_name].aggregate(pipeline,allowDiskUse=True))
end = time.time()
print(end-start)

In [ ]:
print(len(data_list))
print(data_list[0])